In [1]:
# Importation des librairies
import json
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, cross_validate
from xgboost import XGBClassifier

In [2]:
pd.set_option('future.no_silent_downcasting', True)

In [3]:
def read_csv_files_to_dict(base_path, dimension, file_prefix, missing_percentages, strategies=None):
    dataframes_dict = {}

    for percentage in missing_percentages:
        percentage_key = f"{int(percentage * 100)}%"

        if dimension == 'Completeness':
            dataframes_dict[percentage_key] = {}

            if strategies is not None:
                for strategy_name, strategy_func, subfolder in strategies:
                    file_path = f"{base_path}/{dimension}/{subfolder}/{file_prefix}_{percentage_key}.csv"

                    try:
                        df = pd.read_csv(file_path)
                        dataframes_dict[percentage_key][strategy_name] = df
                    except FileNotFoundError:
                        print(f"File not found: {file_path}")
                        dataframes_dict[percentage_key][strategy_name] = None
            else:
                print("No strategies provided for Completeness dimension.")
        elif dimension == 'Unicity':
            file_path = f"{base_path}/{dimension}/{file_prefix}_{percentage_key}_2x.csv"
            try:
                df = pd.read_csv(file_path)
                dataframes_dict[percentage_key] = df
            except FileNotFoundError:
                print(f"File not found: {file_path}")
                dataframes_dict[percentage_key] = None
        else:
            file_path = f"{base_path}/{dimension}/{file_prefix}_{percentage_key}.csv"
            try:
                df = pd.read_csv(file_path)
                dataframes_dict[percentage_key] = df
            except FileNotFoundError:
                print(f"File not found: {file_path}")
                dataframes_dict[percentage_key] = None

    return dataframes_dict

def update_json_results(output_path, model_name, pollution_percentage, results):
    # Charger le fichier JSON existant, ou initialiser une nouvelle structure si le fichier n'existe pas
    if os.path.exists(output_path):
        with open(output_path, 'r') as json_file:
            results_dict = json.load(json_file)
    else:
        results_dict = {
            "models": []
        }

    # Trouver ou ajouter l'entrée pour le modèle spécifié
    model_entry = next((model for model in results_dict["models"] if model["model"] == model_name), None)

    if not model_entry:
        model_entry = {
            "model": model_name,
            "pollution_metrics": []
        }
        results_dict["models"].append(model_entry)

    # Chercher si le pourcentage de pollution existe déjà pour ce modèle
    existing_entry = next((item for item in model_entry["pollution_metrics"] if item["pollution_percentage"] == pollution_percentage), None)

    if existing_entry:
        # Si le pourcentage de pollution existe, remplacer les métriques
        existing_entry["metrics"] = results
    else:
        # Sinon, ajouter une nouvelle entrée pour ce pourcentage
        model_entry["pollution_metrics"].append({
            "pollution_percentage": pollution_percentage,
            "metrics": results
        })

    # Écrire les résultats mis à jour dans le fichier JSON
    with open(output_path, 'w') as json_file:
        json.dump(results_dict, json_file, indent=4)

    print(f"Results saved to {output_path}")

def update_json_results_completeness(output_path, model_name, strategy, pollution_percentage, results):
    # Charger le fichier JSON existant, ou initialiser une nouvelle structure si le fichier n'existe pas
    if os.path.exists(output_path):
        with open(output_path, 'r') as json_file:
            results_dict = json.load(json_file)
    else:
        results_dict = {
            "models": [
                {
                    "model": model_name,
                    "imputation_strategies": []
                }
            ]
        }

    # Trouver ou ajouter l'entrée pour le modèle spécifié
    model_entry = next((model for model in results_dict["models"] if model["model"] == model_name), None)

    if not model_entry:
        model_entry = {
            "model": model_name,
            "imputation_strategies": []
        }
        results_dict["models"].append(model_entry)

    # Trouver ou créer l'entrée pour la stratégie d'imputation actuelle
    strategy_entry = next((item for item in model_entry["imputation_strategies"] if item["strategy"] == strategy), None)

    if not strategy_entry:
        # Ajouter une nouvelle stratégie d'imputation si elle n'existe pas encore
        strategy_entry = {
            "strategy": strategy,
            "pollution_metrics": []
        }
        model_entry["imputation_strategies"].append(strategy_entry)

    # Vérifier si le pourcentage de pollution existe déjà pour cette stratégie
    existing_entry = next((item for item in strategy_entry["pollution_metrics"] if item["pollution_percentage"] == pollution_percentage), None)

    if existing_entry:
        # Si le pourcentage de pollution existe, remplacer les métriques
        existing_entry["metrics"] = results
    else:
        # Sinon, ajouter une nouvelle entrée pour ce pourcentage
        strategy_entry["pollution_metrics"].append({
            "pollution_percentage": pollution_percentage,
            "metrics": results
        })

    # Écrire les résultats mis à jour dans le fichier JSON
    with open(output_path, 'w') as json_file:
        json.dump(results_dict, json_file, indent=4)

    print(f"Results saved to {output_path}")

def prepare_heart_data(heart_df):
    # Remplacer les valeurs de la colonne 'HadHeartAttack' par des booléens
    heart_df['HadHeartAttack'] = heart_df['HadHeartAttack'].replace({'Yes': True, 'No': False}).astype(bool)

    # Convertir les variables catégorielles en variables dummy, en supprimant la première catégorie
    heart_df = pd.get_dummies(heart_df, drop_first=True)

    # Séparer les variables indépendantes (X) de la variable cible (y)
    X = heart_df.drop('HadHeartAttack', axis=1)
    y = heart_df['HadHeartAttack']

    return X, y

def train_and_evaluate(X, y):
    model = XGBClassifier()
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    scoring = ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']
    cv_results = cross_validate(model, X, y, cv=kfold, scoring=scoring)

    results = {
        "accuracy": cv_results['test_accuracy'].mean(),
        "precision": cv_results['test_precision_macro'].mean(),
        "recall": cv_results['test_recall_macro'].mean(),
        "f1_score": cv_results['test_f1_macro'].mean()
    }

    return results

## Completeness

In [30]:
# Utilisation de la fonction dans votre code existant
missing_percentages = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]

# Strategy :
strategy_delete = {
    'numerical': 'delete',
    'categorical': 'delete'
}

strategy_mean_mode = {
    'numerical': 'mean',
    'categorical': 'mode'
}

strategy_median_new = {
    'numerical': 'median',
    'categorical': 'new'
}

strategy_decision_tree = {
    'numerical': 'decision_tree',
    'categorical': 'decision_tree'
}

strategy_mean_new = {
    'numerical': 'mean',
    'categorical': 'new'
}

strategy_knn_mode = {
    'numerical': 'knn',
    'categorical': 'mode'
}

strategies = [
    ("Mean and Mode", strategy_mean_mode, "Mean and Mode"),
    ("Median and New", strategy_median_new, "Median and New"),
    ("Decision Tree", strategy_decision_tree, "Decision Tree"),
    ("Mean and New", strategy_mean_new, "Mean and New")
]

heart_df_dict = read_csv_files_to_dict('../../Data/Classification/Indicators of Heart Disease', 'Completeness', 'heart', missing_percentages, strategies)

# Chemin du fichier JSON où les résultats seront enregistrés
output_path = "../../Results/Heart Disease/Completeness.json"

# Boucle sur les DataFrames pour chaque pourcentage de pollution
for key, heart_df_strategies in heart_df_dict.items():
    for strategy_name, heart_df in heart_df_strategies.items():
        X, y = prepare_heart_data(heart_df)

        results = train_and_evaluate(X, y)

        # Extraire le pourcentage de pollution à partir de la clé
        pollution_percentage = float(key.replace('%', ''))

        # Mise à jour des résultats dans le fichier JSON en utilisant la fonction
        update_json_results_completeness(output_path, "XGBoost", strategy_name, pollution_percentage, results)

0%
Mean and Mode
(246022, 122)
(246022,)
{'strategy': 'Mean and Mode', 'pollution_metrics': [{'pollution_percentage': 0, 'metrics': {'accuracy': 0.9478867709283898, 'precision': 0.7546709544012085, 'recall': 0.6166185651988428, 'f1_score': 0.6559323267381035}}]}
Median and New
(246022, 122)
(246022,)
{'strategy': 'Median and New', 'pollution_metrics': [{'pollution_percentage': 0, 'metrics': {'accuracy': 0.9478867709283898, 'precision': 0.7546709544012085, 'recall': 0.6166185651988428, 'f1_score': 0.6559323267381035}}]}
Decision Tree
(246022, 122)
(246022,)
{'strategy': 'Decision Tree', 'pollution_metrics': [{'pollution_percentage': 0, 'metrics': {'accuracy': 0.9478867709283898, 'precision': 0.7546709544012085, 'recall': 0.6166185651988428, 'f1_score': 0.6559323267381035}}]}
Mean and New
(246022, 122)
(246022,)
{'strategy': 'Mean and New', 'pollution_metrics': [{'pollution_percentage': 0, 'metrics': {'accuracy': 0.9478867709283898, 'precision': 0.7546709544012085, 'recall': 0.6166185651

## Consistent Representation

In [12]:
# Utilisation de la fonction dans votre code existant
pollution_percentage_levels = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
heart_df_dict = read_csv_files_to_dict('../../Data/Classification/Indicators of Heart Disease', 'Consistent Representation', 'heart', pollution_percentage_levels)

# Chemin du fichier JSON où les résultats seront enregistrés
output_path = "../../Results/Heart Disease/Consistent Representation.json"

# Boucle sur les DataFrames pour chaque pourcentage de pollution
for key, heart_df in heart_df_dict.items():
    X, y = prepare_heart_data(heart_df)

    results = train_and_evaluate(X, y)

    # Extraire le pourcentage de pollution à partir de la clé
    pollution_percentage = float(key.replace('%', ''))

    # Mise à jour des résultats dans le fichier JSON
    update_json_results(output_path, "XGBoost", pollution_percentage, results)

Results saved to ../../Results/Heart Disease/Consistent Representation.json
Results saved to ../../Results/Heart Disease/Consistent Representation.json
Results saved to ../../Results/Heart Disease/Consistent Representation.json
Results saved to ../../Results/Heart Disease/Consistent Representation.json
Results saved to ../../Results/Heart Disease/Consistent Representation.json
Results saved to ../../Results/Heart Disease/Consistent Representation.json
Results saved to ../../Results/Heart Disease/Consistent Representation.json
Results saved to ../../Results/Heart Disease/Consistent Representation.json
Results saved to ../../Results/Heart Disease/Consistent Representation.json


## Feature Accuracy

In [ ]:
pollution_percentage_levels = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
heart_df_dict = read_csv_files_to_dict('../../Data/Classification/Indicators of Heart Disease', 'Feature Accuracy', 'heart', pollution_percentage_levels)

# Chemin du fichier JSON où les résultats seront enregistrés
output_path = "../../Results/Heart Disease/Feature Accuracy.json"

# Boucle sur les DataFrames pour chaque pourcentage de pollution
for key, heart_df in heart_df_dict.items():
    X, y = prepare_heart_data(heart_df)

    results = train_and_evaluate(X, y)

    # Extraire le pourcentage de pollution à partir de la clé
    pollution_percentage = float(key.replace('%', ''))

    # Mise à jour des résultats dans le fichier JSON
    update_json_results(output_path, "XGBoost", pollution_percentage, results)

## Target Accuracy

In [9]:
pollution_percentage_levels = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
heart_df_dict = read_csv_files_to_dict('../../Data/Classification/Indicators of Heart Disease', 'Target Accuracy', 'heart', pollution_percentage_levels)

# Chemin du fichier JSON où les résultats seront enregistrés
output_path = "../../Results/Heart Disease/Target Accuracy.json"

# Boucle sur les DataFrames pour chaque pourcentage de pollution
for key, heart_df in heart_df_dict.items():
    X, y = prepare_heart_data(heart_df)

    results = train_and_evaluate(X, y)

    # Extraire le pourcentage de pollution à partir de la clé
    pollution_percentage = float(key.replace('%', ''))

    # Mise à jour des résultats dans le fichier JSON
    update_json_results(output_path, "XGBoost", pollution_percentage, results)

Results saved to ../../Results/Heart Disease/Target Accuracy.json


## Target Class Balance

In [10]:
balance_levels = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
heart_df_dict = read_csv_files_to_dict('../../Data/Classification/Indicators of Heart Disease', 'Target Class Balance', 'heart', balance_levels)

# Chemin du fichier JSON où les résultats seront enregistrés
output_path = "../../Results/Heart Disease/Target Class Balance.json"

# Boucle sur les DataFrames pour chaque pourcentage de pollution
for key, heart_df in heart_df_dict.items():
    X, y = prepare_heart_data(heart_df)

    results = train_and_evaluate(X, y)

    # Extraire le pourcentage de pollution à partir de la clé
    pollution_percentage = float(key.replace('%', ''))

    # Mise à jour des résultats dans le fichier JSON
    update_json_results(output_path, "XGBoost", pollution_percentage, results)

Results saved to ../../Results/Heart Disease/Target Class Balance.json


## Unicity

In [11]:
pollution_percentage_levels = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
heart_df_dict = read_csv_files_to_dict('../../Data/Classification/Indicators of Heart Disease', 'Unicity', 'heart', pollution_percentage_levels)

# Chemin du fichier JSON où les résultats seront enregistrés
output_path = "../../Results/Heart Disease/Unicity.json"

# Boucle sur les DataFrames pour chaque pourcentage de pollution
for key, heart_df in heart_df_dict.items():
    X, y = prepare_heart_data(heart_df)

    results = train_and_evaluate(X, y)

    # Extraire le pourcentage de pollution à partir de la clé
    pollution_percentage = float(key.replace('%', ''))

    # Mise à jour des résultats dans le fichier JSON
    update_json_results(output_path, "XGBoost", pollution_percentage, results)

Results saved to ../../Results/Heart Disease/Unicity.json
